In [1]:
def read_input(filename, split = False, convert_to_int = False, sep = '\n'):
    f = open(filename)
    raw = f.read()[:-1]
    f.close()
    data = raw
    if split:
        data = data.split(sep)
    if convert_to_int:
        data = [int(item) for item in data]
    return data

In [2]:
sample_data, data = read_input('sample.txt'), read_input('input.txt')

In [201]:
class PolymerizationMachine:
    def __init__(self, data):
        self.template = data.split('\n\n')[0]
        self.insertion_rules = {}
        for rule in data.split('\n\n')[1].split('\n'):
            self.insertion_rules[rule.split(' -> ')[0]] = rule.split(' -> ')[1]
        self.formula = self.template
        
        self.pairs_counter = {}
        for key in self.insertion_rules:
            self.pairs_counter[key] = 0
        for i in range(1,len(self.template)):
            self.pairs_counter[self.template[i-1:i+1]] += 1
            
        self.element_freqs = {}
        for s in set(''.join(self.pairs_counter.keys())):
            self.element_freqs[s] = 0
        for s in self.template:
            self.element_freqs[s] += 1
        
    def take_step(self):
        new_formula = ''
        for i in range(1,len(self.formula)):
            if self.formula[i-1:i+1] in self.insertion_rules:
                new_formula += self.formula[i-1] + self.insertion_rules[self.formula[i-1:i+1]]
        new_formula += self.formula[-1]
        self.formula = new_formula
    
    def run(self, n):
        for i in range(n):
            self.take_step()
        frequencies = []
        for s in set(self.formula):
            frequencies.append(self.formula.count(s))
        return max(frequencies) - min(frequencies)
    
    def take_smart_step(self):
        add = {}
        for key in self.pairs_counter:
            if self.pairs_counter[key] > 0:
                self.element_freqs[self.insertion_rules[key]] += self.pairs_counter[key]
                if key[0]+self.insertion_rules[key] in add:
                    add[key[0]+self.insertion_rules[key]] += self.pairs_counter[key]
                else:
                    add[key[0]+self.insertion_rules[key]] = self.pairs_counter[key]
                if self.insertion_rules[key]+key[1] in add:
                    add[self.insertion_rules[key]+key[1]] += self.pairs_counter[key]
                else:
                    add[self.insertion_rules[key]+key[1]] = self.pairs_counter[key]
                if key in add:
                    add[key] -= self.pairs_counter[key]
                else:
                    add[key] = -1*self.pairs_counter[key]
        for key in add:
            self.pairs_counter[key] += add[key]
        
    def smart_run(self, n):
        for i in range(n):
            self.take_smart_step()
        return max(self.element_freqs.values())-min(self.element_freqs.values())

In [183]:
PolymerizationMachine(sample_data).run(10) == 1588

True

In [209]:
%%time
PolymerizationMachine(data).run(10)

CPU times: user 22 ms, sys: 1.98 ms, total: 24 ms
Wall time: 22.4 ms


3306

In [211]:
%%time
PolymerizationMachine(data).smart_run(10)

CPU times: user 1.7 ms, sys: 2 µs, total: 1.7 ms
Wall time: 1.71 ms


3306

In [207]:
%%time
PolymerizationMachine(sample_data).smart_run(40) == 2188189693529

CPU times: user 1.44 ms, sys: 3 µs, total: 1.44 ms
Wall time: 1.44 ms


True

In [210]:
%%time
PolymerizationMachine(data).smart_run(40)

CPU times: user 7.25 ms, sys: 471 µs, total: 7.72 ms
Wall time: 7.3 ms


3760312702877